<a href="https://colab.research.google.com/github/Bhavya-2k03/YOLO/blob/main/CustomTrainYOLOv5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import xml.etree.ElementTree as ET
from pathlib import Path

In [ ]:
!pip install -U --no-cache-dir gdown --pre 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [ ]:
%mkdir yolov5_ws 
%cd yolov5_ws 
!git clone https://github.com/ultralytics/yolov5 
%cd yolov5 
%pip install -r requirements.txt

/content/yolov5_ws
Cloning into 'yolov5'...
remote: Enumerating objects: 14918, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 14918 (delta 2), reused 5 (delta 0), pack-reused 14908
Receiving objects: 100% (14918/14918), 13.96 MiB | 17.25 MiB/s, done.
Resolving deltas: 100% (10249/10249), done.
/content/yolov5_ws/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 184 kB 7.7 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 
     |████████████████████████████████| 1.6 MB 84.9 MB/s 


In [ ]:
%cd -

/content/yolov5_ws


In [ ]:
!gdown --id 1qM_xcCK_USEHjsCoPdH8zFtugQPw0Cun

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1qM_xcCK_USEHjsCoPdH8zFtugQPw0Cun
To: /content/yolov5_ws/m2cai16-tool-locations.zip
100% 145M/145M [00:00<00:00, 151MB/s]


In [ ]:
!unzip m2cai16-tool-locations.zip

Streaming output truncated to the last 5000 lines.
  inflating: m2cai16-tool-locations/JPEGImages/v06_042450_flip.jpg  
  inflating: __MACOSX/m2cai16-tool-locations/JPEGImages/._v06_042450_flip.jpg  
  inflating: m2cai16-tool-locations/JPEGImages/v06_042475.jpg  
  inflating: __MACOSX/m2cai16-tool-locations/JPEGImages/._v06_042475.jpg  
  inflating: m2cai16-tool-locations/JPEGImages/v06_042475_flip.jpg  
  inflating: __MACOSX/m2cai16-tool-locations/JPEGImages/._v06_042475_flip.jpg  
  inflating: m2cai16-tool-locations/JPEGImages/v06_042500.jpg  
  inflating: __MACOSX/m2cai16-tool-locations/JPEGImages/._v06_042500.jpg  
  inflating: m2cai16-tool-locations/JPEGImages/v06_042525.jpg  
  inflating: __MACOSX/m2cai16-tool-locations/JPEGImages/._v06_042525.jpg  
  inflating: m2cai16-tool-locations/JPEGImages/v06_042525_flip.jpg  
  inflating: __MACOSX/m2cai16-tool-locations/JPEGImages/._v06_042525_flip.jpg  
  inflating: m2cai16-tool-locations/JPEGImages/v06_042550.jpg  
  inflating: __MACOSX

In [ ]:
%cd m2cai16-tool-locations

/content/yolov5_ws/m2cai16-tool-locations


In [ ]:
import xml.etree.ElementTree as ET
import glob
import os
import json


def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]


def yolo_to_xml_bbox(bbox, w, h):
    # x_center, y_center width heigth
    w_half_len = (bbox[2] * w) / 2
    h_half_len = (bbox[3] * h) / 2
    xmin = int((bbox[0] * w) - w_half_len)
    ymin = int((bbox[1] * h) - h_half_len)
    xmax = int((bbox[0] * w) + w_half_len)
    ymax = int((bbox[1] * h) + h_half_len)
    return [xmin, ymin, xmax, ymax]


classes = []
input_dir = "Annotations/"
output_dir = "labels/"
image_dir = "JPEGImages/"

# create the labels folder (output directory)
os.mkdir(output_dir)

# identify all the xml files in the annotations folder (input directory)
files = glob.glob(os.path.join(input_dir, '*.xml'))
# loop through each 
for fil in files:
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    # check if the label contains the corresponding image file
    if not os.path.exists(os.path.join(image_dir, f"{filename}.jpg")):
        print(f"{filename} image does not exist!")
        continue

    result = []

    # parse the content of the xml file
    tree = ET.parse(fil)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)

    for obj in root.findall('object'):
        label = obj.find("name").text
        # check for new classes and append to list
        if label not in classes:
            classes.append(label)
        index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
        # convert data to string
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{index} {bbox_string}")

    if result:
        # generate a YOLO format text file for each xml file
        with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding="utf-8") as f:
            f.write("\n".join(result))

# generate the classes file as reference
with open('classes.txt', 'w', encoding='utf8') as f:
    f.write(json.dumps(classes))

In [ ]:
%cd -

/content/yolov5_ws


In [ ]:
allfiles=[]
allfiles.append(os.listdir("/content/yolov5_ws/m2cai16-tool-locations/JPEGImages/"))
import re
count=0
for f in allfiles:
  for y in f:
    if re.search('flip',y):
      # print(y)
      os.remove(f"m2cai16-tool-locations/JPEGImages/{y}")
      count=count+1
print(count)

2248


In [ ]:
len(os.listdir("/content/yolov5_ws/m2cai16-tool-locations/JPEGImages/"))

2811

In [ ]:
if os.path.exists('/content/yolov5_ws/data'):
    pass
else:
    os.mkdir('/content/yolov5_ws/data')

In [ ]:
import os, shutil, random

# preparing the folder structure

full_data_path_images = 'm2cai16-tool-locations/JPEGImages/'
full_data_path_labels = 'm2cai16-tool-locations/labels/'
extension_allowed = '.jpg'
split_percentage = 90

images_path = 'data/images/'
if os.path.exists(images_path):
    shutil.rmtree(images_path)
os.mkdir(images_path)
    
labels_path = 'data/labels/'
if os.path.exists(labels_path):
    shutil.rmtree(labels_path)
os.mkdir(labels_path)
    
training_images_path = images_path + 'training/'
validation_images_path = images_path + 'validation/'
training_labels_path = labels_path + 'training/'
validation_labels_path = labels_path +'validation/'
    
os.mkdir(training_images_path)
os.mkdir(validation_images_path)
os.mkdir(training_labels_path)
os.mkdir(validation_labels_path)

files = []

ext_len = len(extension_allowed)

for r, d, f in os.walk(full_data_path_images):
    for file in f:
        if file.endswith(extension_allowed):
            strip = file[0:len(file) - ext_len]      
            files.append(strip)

random.shuffle(files)

size = len(files)                   

split = int(split_percentage * size / 100)

for i in range(split):
    strip = files[i]
                         
    image_file = strip + extension_allowed
    src_image = full_data_path_images + image_file
    shutil.copy(src_image, training_images_path) 
                         
    annotation_file = strip + '.txt'
    src_label = full_data_path_labels + annotation_file
    shutil.copy(src_label, training_labels_path) 

for i in range(split, size):
    strip = files[i]
                         
    image_file = strip + extension_allowed
    src_image = full_data_path_images + image_file
    shutil.copy(src_image, validation_images_path) 
                         
    annotation_file = strip + '.txt'
    src_label = full_data_path_labels + annotation_file
    shutil.copy(src_label, validation_labels_path) 


In [ ]:
with open('m2cai16-tool-locations/classes.txt','r') as firstfile, open('dataset.yaml','w') as secondfile:
    for line in firstfile:
            secondfile.write(f'''train: ../data/images/training/
val: ../data/images/validation/

# number of classes
nc: 7

# class names
names: {line}''')

In [ ]:
!python yolov5/train.py --img 416 --batch 4 --epochs 100 --data dataset.yaml --weights yolov5s.pt
print("done")

train: weights=yolov5s.pt, cfg=, data=dataset.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=4, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-55-g632bf48 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.

In [ ]:
!python yolov5/detect.py --weights yolov5/runs/train/exp2/weights/best.pt --img 415 --conf 0.5 --source data/images/training/v01_002250.jpg

detect: weights=['yolov5/runs/train/exp2/weights/best.pt'], source=data/images/training/v01_002250.jpg, data=yolov5/data/coco128.yaml, imgsz=[415, 415], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-55-g632bf48 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 157 layers, 7029004 parameters, 0 gradients, 15.8 GFLOPs
WARNING ⚠️ --img-size [415, 415] must be multiple of max stride 32, updating to [416, 416]
image 1/1 /content/yolov5_ws/data/images/training/v01_002250.jpg: 256x416 2 Graspers, 10.7ms
Speed: 0.3ms pre-process, 10.7ms inference, 1.5ms NMS per image at shape (1, 3, 416, 416)
Results saved to yolov5/runs/detect/